In [37]:
import pandas as pd
df=pd.read_csv('data/Original/healthcare_dataset.csv')
df=df[: 1000].applymap(lambda x: x.lower() if type(x) == str else x)
df

/tmp/ipykernel_544011/3386679760.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df=df[: 1000].applymap(lambda x: x.lower() if type(x) == str else x)


,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,bobby jackson,30,male,b-,cancer,2024-01-31,matthew smith,sons and miller,blue cross,18856.281306,328,urgent,2024-02-02,paracetamol,normal
1,leslie terry,62,male,a+,obesity,2019-08-20,samantha davies,kim inc,medicare,33643.327287,265,emergency,2019-08-26,ibuprofen,inconclusive
2,danny smith,76,female,a-,obesity,2022-09-22,tiffany mitchell,cook plc,aetna,27955.096079,205,emergency,2022-10-07,aspirin,normal
3,andrew watts,28,female,o+,diabetes,2020-11-18,kevin wells,"hernandez rogers and vang,",medicare,37909.782410,450,elective,2020-12-18,ibuprofen,abnormal
4,adrienne bell,43,female,ab+,cancer,2022-09-19,kathleen hanna,white-white,aetna,14238.317814,458,urgent,2022-10-09,penicillin,abnormal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,timothy diaz,25,female,b-,diabetes,2021-10-24,michael grant,jones-ellis,medicare,44041.573738,454,emergency,2021-11-13,ibuprofen,normal
996,daniel blankenship,66,female,b+,arthritis,2023-05-24,debra carpenter,"jenkins, mathews and ho",medicare,15086.158780,445,urgent,2023-06-09,lipitor,abnormal
997,mary stephens,62,male,a+,hypertension,2019-12-20,amanda johnson,villa inc,unitedhealthcare,16377.523128,174,urgent,2020-01-17,penicillin,inconclusive
998,brian figueroa,30,male,a-,hypertension,2022-04-27,joseph flores,castro plc,unitedhealthcare,43488.838627,341,elective,2022-05-04,lipitor,inconclusive


In [43]:
df.nunique()

Name                   997
Age                     68
Gender                   2
Blood Type               8
Medical Condition        6
Date of Admission      780
Doctor                 992
Hospital               988
Insurance Provider       5
Billing Amount        1000
Room Number            360
Admission Type           3
Discharge Date         764
Medication               5
Test Results             3
dtype: int64

In [44]:
df['Name'].nunique()
df.loc[df['Name']=='michael smith']

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
161,michael smith,66,female,ab-,obesity,2022-06-03,austin jackson,"morgan rios and keller,",cigna,33861.018632,115,elective,2022-06-27,aspirin,normal
235,michael smith,20,male,ab+,arthritis,2020-03-27,eddie jackson,"huang travis thomas, and",unitedhealthcare,2784.582490,164,emergency,2020-04-16,aspirin,inconclusive


In [45]:
template='''[Name], a [Age]-year-old [Gender] with [Blood Type] blood, was admitted to [Hospital] on [Date of Admission] due to [Medical Condition]. Under the care of Dr. [Doctor], the patient was assigned to Room [Room Number] for [Admission Type] treatment. Their insurance provider, [Insurance Provider], covered a billing amount of [Billing Amount] dollars. The patient was discharged on [Discharge Date] after receiving [Medication], with test results showing [Test Results].'''

In [46]:
def generate_sentence(i):
    row = df.iloc[i].fillna("N/A")  # Fill NaN values with "N/A"
    return template.replace('[Name]', row['Name'].lower()) \
                   .replace('[Age]', str(row['Age'])) \
                   .replace('[Gender]', row['Gender']) \
                   .replace('[Blood Type]', row['Blood Type']) \
                   .replace('[Hospital]', row['Hospital'].lower()) \
                   .replace('[Date of Admission]', row['Date of Admission']) \
                   .replace('[Medical Condition]', row['Medical Condition'].lower()) \
                   .replace('[Doctor]', row['Doctor'].lower()) \
                   .replace('[Room Number]', str(row['Room Number'])) \
                   .replace('[Admission Type]', row['Admission Type'].lower()) \
                   .replace('[Insurance Provider]', row['Insurance Provider'].lower()) \
                   .replace('[Billing Amount]', str(row['Billing Amount'])) \
                   .replace('[Discharge Date]', row['Discharge Date']) \
                   .replace('[Medication]', row['Medication'].lower()) \
                   .replace('[Test Results]', row['Test Results'])    

healthcare_report = [generate_sentence(i) for i in range(len(df))]
healthcare_report

['bobby jackson, a 30-year-old male with b- blood, was admitted to sons and miller on 2024-01-31 due to cancer. Under the care of Dr. matthew smith, the patient was assigned to Room 328 for urgent treatment. Their insurance provider, blue cross, covered a billing amount of 18856.281305978155 dollars. The patient was discharged on 2024-02-02 after receiving paracetamol, with test results showing normal.',
 'leslie terry, a 62-year-old male with a+ blood, was admitted to kim inc on 2019-08-20 due to obesity. Under the care of Dr. samantha davies, the patient was assigned to Room 265 for emergency treatment. Their insurance provider, medicare, covered a billing amount of 33643.327286577885 dollars. The patient was discharged on 2019-08-26 after receiving ibuprofen, with test results showing inconclusive.',
 'danny smith, a 76-year-old female with a- blood, was admitted to cook plc on 2022-09-22 due to obesity. Under the care of Dr. tiffany mitchell, the patient was assigned to Room 205 fo

In [48]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import simpleSplit

def create_pdf(sentences, filename="health_dataset.pdf"):
    # Set up the PDF canvas
    pdf = canvas.Canvas(filename, pagesize=letter)
    width, height = letter
    
    # Set margins
    x = 50  # Left margin
    y = height - 50  # Start from the top of the page
    max_width = width - 100  # Right margin (50px padding)
    
    # Define font
    pdf.setFont("Helvetica", 12)

    # Loop through sentences and add them to the PDF
    for sentence in sentences:
        # Wrap text within max width
        wrapped_lines = simpleSplit(sentence, "Helvetica", 12, max_width)

        for line in wrapped_lines:
            pdf.drawString(x, y, line)
            y -= 20  # Move down for the next line
            
            # If we run out of space, create a new page
            if y < 50:
                pdf.showPage()
                pdf.setFont("Helvetica", 12)  # Reset font on new page
                y = height - 50  # Reset y position

    # Save the PDF
    pdf.save()
    print(f"PDF saved as {filename}")



# Create the PDF
create_pdf(healthcare_report, "data/preprocessed-1k-samples/patient_report.pdf")


PDF saved as data/preprocessed-1k-samples/patient_report.pdf


In [50]:
df.to_csv('data/preprocessed-1k-samples/CSV/Med_recoreds.csv', index=False)